In [1]:
using TextAnalysis, CorpusLoaders, MultiResolutionIterators, LinearAlgebra

┌ Info: Recompiling stale cache file /home/ayushk4/.julia/compiled/v1.0/TextAnalysis/5Mwet.ji for TextAnalysis [a2db99b7-8b79-58f8-94bf-bbc811eef33d]
└ @ Base loading.jl:1190
┌ Warning: Package TextAnalysis does not have Libdl in its dependencies:
│ - If you have TextAnalysis checked out for development and have
│   added Libdl as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with TextAnalysis
└ Loading Libdl into TextAnalysis from project dependency, future warnings for TextAnalysis are suppressed.


In [2]:
ner = NERTagger()

TextAnalysis.NERmodel{TextAnalysis.BiLSTM_CNN_CRF_Model{Flux.Conv{2,2,typeof(identity),TrackedArray{…,Array{Float32,4}},TrackedArray{…,Array{Float32,1}}},Array{Float32,2},Flux.Recur{Flux.LSTMCell{TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}}},Flux.Dense{typeof(identity),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},CRF{TrackedArray{…,Array{Float32,2}}},Array{Int64,2}}}(TextAnalysis.BiLSTM_CNN_CRF_Model{Flux.Conv{2,2,typeof(identity),TrackedArray{…,Array{Float32,4}},TrackedArray{…,Array{Float32,1}}},Array{Float32,2},Flux.Recur{Flux.LSTMCell{TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}}},Flux.Dense{typeof(identity),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},CRF{TrackedArray{…,Array{Float32,2}}},Array{Int64,2}}(["ORG", "O", "MISC", "PER", "PER", "LOC", "ORG", "ORG", "PER", "PER", "MISC", "MISC", "MISC", "ORG", "LOC", "LOC", "LOC"], Dict('w'=>81,'E'=>34,'7'=>22,'Z'=>55,']'=>57,'o'=>73,'B'=>31,'5'=>20,'h'

In [3]:
test_set = CorpusLoaders.load(CoNLL(), "test") # test set
test_dataset = flatten_levels(test_set, lvls(CoNLL, :document)) |> full_consolidate

CorpusLoaders.Document{Array{Array{CorpusLoaders.NERTaggedWord,1},1},String}("test.txt", Array{CorpusLoaders.NERTaggedWord,1}[[NERTaggedWord("O", "B-NP", "NN", "SOCCER"), NERTaggedWord("O", "O", ":", "-"), NERTaggedWord("B-LOC", "B-NP", "NNP", "JAPAN"), NERTaggedWord("O", "B-VP", "VB", "GET"), NERTaggedWord("O", "B-NP", "NNP", "LUCKY"), NERTaggedWord("O", "I-NP", "NNP", "WIN"), NERTaggedWord("O", "O", ",", ","), NERTaggedWord("B-PER", "B-NP", "NNP", "CHINA"), NERTaggedWord("O", "B-PP", "IN", "IN"), NERTaggedWord("O", "B-NP", "DT", "SURPRISE"), NERTaggedWord("O", "I-NP", "NN", "DEFEAT"), NERTaggedWord("O", "O", ".", ".")], [NERTaggedWord("B-PER", "B-NP", "NNP", "Nadim"), NERTaggedWord("I-PER", "I-NP", "NNP", "Ladki")], [NERTaggedWord("B-LOC", "B-NP", "NNP", "AL-AIN"), NERTaggedWord("O", "O", ",", ","), NERTaggedWord("B-LOC", "B-NP", "NNP", "United"), NERTaggedWord("I-LOC", "I-NP", "NNP", "Arab"), NERTaggedWord("I-LOC", "I-NP", "NNPS", "Emirates"), NERTaggedWord("O", "I-NP", "CD", "1996-

In [4]:
function obtain_X_Y(dataset)
    X = [CorpusLoaders.word.(sent) for sent in dataset]
    Y = [TextAnalysis.remove_ner_label_prefix.(CorpusLoaders.named_entity.(sent)) for sent in dataset]
    return X, Y
end

X, Y = obtain_X_Y(test_dataset)

(Array{String,1}[["SOCCER", "-", "JAPAN", "GET", "LUCKY", "WIN", ",", "CHINA", "IN", "SURPRISE", "DEFEAT", "."], ["Nadim", "Ladki"], ["AL-AIN", ",", "United", "Arab", "Emirates", "1996-12-06"], ["Japan", "began", "the", "defence", "of", "their", "Asian", "Cup", "title", "with"  …  "Syria", "in", "a", "Group", "C", "championship", "match", "on", "Friday", "."], ["But", "China", "saw", "their", "luck", "desert", "them", "in", "the", "second"  …  "crashing", "to", "a", "surprise", "2-0", "defeat", "to", "newcomers", "Uzbekistan", "."], ["China", "controlled", "most", "of", "the", "match", "and", "saw", "several", "chances"  …  "the", "advancing", "Chinese", "keeper", "and", "into", "an", "empty", "net", "."], ["Oleg", "Shatskiku", "made", "sure", "of", "the", "win", "in", "injury", "time"  …  "unstoppable", "left", "foot", "shot", "from", "just", "outside", "the", "area", "."], ["The", "former", "Soviet", "republic", "was", "playing", "in", "an", "Asian", "Cup", "finals", "tie", "for", "t

In [151]:
function eval_ner_tagger(ner_m, x_seq, y_seq, unique_labels, confusion_matrix)
    preds = ner_m(x_seq)

    for (pred, logit) in zip(preds, y_seq)
        logit == "MISC" && continue
        confusion_matrix[findfirst(x -> x==pred, unique_labels), findfirst(x -> x==logit, unique_labels)] += 1
    end
end

function try_outs(ner_m, x_in, y_in, eval_func)
    unique_labels = unique(ner.model.labels)
    num_labels = length(unique_labels)
    confusion_matrix = zeros(Int, (num_labels, num_labels))

    for (x_seq, y_seq) in zip(x_in, y_in)

    eval_func(ner_m, x_seq, y_seq, unique_labels, confusion_matrix)
#         preds = ner(x_seq)

#         for (pred, logit) in zip(preds, y_seq)
#             logit == "MISC" && continue
#             confusion_matrix[findfirst(x -> x==pred, unique_labels), findfirst(x -> x==logit, unique_labels)] += 1
#         end
    end

    s1 = sum(confusion_matrix, dims=2)
    s2 = sum(confusion_matrix, dims=1)'
    dg = diag(confusion_matrix)
    s1 = [s1[1:2]..., s1[4:5]...]
    s2 = [s2[1:2]..., s2[4:5]...]
    dg = [dg[1:2]..., dg[4:5]...]

    unique_labels = unique(ner.model.labels)
    deleteat!(unique_labels, findfirst(x -> x=="MISC", unique_labels))
    # Don't count MISC
    
    f1s = []

    for (p, r, d, tag) in zip(s1, s2, dg, unique_labels)
        println("For tag `$tag`")
        prec = d/p
        recall = d/r
        f1 = (2 * prec * recall) /(prec + recall)
        println("The precision is $prec")
        println("The recall is $recall")
        println("f1 is $f1")
        println()
        push!(f1s, f1)
    end

    a = sum(dg ./ s1) / length(unique_labels)
    b = sum(dg ./ s2) / length(unique_labels)
    println("Overall Micro f1 for NER (excluding MISC) on CoNLL 2003 is ", (2 * a * b)/ (a + b))
    println("Overall Macro f1 for NER (excluding MISC) on CoNLL 2003 is ", sum(f1s)/ length(f1s))
end

try_outs (generic function with 1 method)

In [9]:
try_outs(ner, X, Y, eval_ner_tagger)

For tag `ORG`
The precision is 0.8897058823529411
The recall is 0.8241185897435898
f1 is 0.855657237936772

For tag `O`
The precision is 0.9901429018462501
The recall is 0.9907888213344467
f1 is 0.9904657562833405

For tag `PER`
The precision is 0.9708029197080292
The recall is 0.9592499098449333
f1 is 0.9649918374750589

For tag `LOC`
The precision is 0.9051504334523203
The recall is 0.922077922077922
f1 is 0.9135357694287184

Overall Micro f1 for NER (excluding MISC) on CoNLL 2003 is 0.9314451550144375
Overall Macro f1 for NER (excluding MISC) on CoNLL 2003 is 0.9311626502809724


In [85]:
using PyCall, WordTokenizers
spacy = pyimport("spacy")
nlp = spacy.load("en_core_web_sm")

PyObject <spacy.lang.en.English object at 0x7f3064a39240>

In [150]:
function eval_spacy_tagger(ner_m, x_seq, y_seq, unique_labels, confusion_matrix)
    preds = String[]
    ents = nlp(join(x_seq, " ")).ents

    idx = 1
    i = 1
    while i <= length(x_seq)
        if idx <= length(ents) && x_seq[i] == tokenize(ents[idx].text)[1]
            l = length(tokenize(ents[idx].text))
            for k in 1:l
                pred = ents[idx].label_
                if (pred == "PERSON")
                    push!(preds, "PER")
                elseif (pred == "GPE" || pred == "LOC")
                    push!(preds, "LOC")
                elseif (pred == "ORG")
                    push!(preds, pred)
                else
                    push!(preds, "MISC")
                end
            end
            i = i + l - 1
            idx += 1
        else
            push!(preds, "O")
        end
        println(x_seq[i])
        i += 1
    end
    @assert length(preds) == length(y_seq)

    for (pred, logit) in zip(preds, y_seq)
        (logit == "MISC") && continue
        confusion_matrix[findfirst(x -> x==pred, unique_labels), findfirst(x -> x==logit, unique_labels)] += 1
    end
end


eval_ner_tagger (generic function with 1 method)

7-element Array{String,1}:
 "PER"
 "PER"
 "O"  
 "ORG"
 "O"  
 "O"  
 "O"  

In [145]:
x_seq

7-element Array{String,1}:
 "Ayush"   
 "Kaushal" 
 "from"    
 "Dehradun"
 "is"      
 "here"    
 "."       

In [110]:
oo.label_

"LOC"